# ML - Problem Set 2: Predicting Poverty

El objetivo principal es construir un modelo predictivo de la pobreza de los hogares. Nótese que un hogar se clasifica como

\begin{equation}
    Poor = I \left( Inc < Pl \right)
\end{equation}

donde $I$ es una función indicadora que toma uno si el ingreso familiar está por debajo de una determinada línea de pobreza.

In [1]:
# Importar librerias
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
# Importar datos

df_1 = pd.read_csv('sample_submission.csv')
df_2 = pd.read_csv('test_hogares.csv')
df_3 = pd.read_csv('test_personas.csv')
df_4 = pd.read_csv('train_hogares.csv')
df_5 = pd.read_csv('train_personas.csv')

In [3]:
# Renombrar columnas antes de combinar
df_4 = df_4.rename(columns={'Clase': 'Clase_hogar', 'Dominio': 'Dominio_hogar'})
df_5 = df_5.rename(columns={'Clase': 'Clase_persona', 'Dominio': 'Dominio_persona'})

# Luego combinar
train_combined = pd.merge(df_4, df_5, on='id', how='inner')

# Combinar datos de prueba
test_combined = pd.merge(df_2, df_3, on='id', how='inner')

In [4]:
print("Dimensiones del dataset combinado de entrenamiento:", train_combined.shape)
print("Dimensiones del dataset combinado de prueba:", test_combined.shape)

Dimensiones del dataset combinado de entrenamiento: (543109, 157)
Dimensiones del dataset combinado de prueba: (219644, 78)


In [5]:
# Hogares sin personas asociadas
hogares_sin_personas = df_4[~df_4['id'].isin(df_5['id'])]
print("Número de hogares sin personas asociadas:", len(hogares_sin_personas))

# Personas sin hogares asociados
personas_sin_hogares = df_5[~df_5['id'].isin(df_4['id'])]
print("Número de personas sin hogares asociados:", len(personas_sin_hogares))

Número de hogares sin personas asociadas: 0
Número de personas sin hogares asociados: 0


In [6]:
# Identificar columnas con valores faltantes
missing_values = train_combined.isnull().sum()
print("Valores faltantes por columna:\n", missing_values)

Valores faltantes por columna:
 id                    0
Clase_hogar           0
Dominio_hogar         0
P5000                 0
P5010                 0
                  ...  
Ingtotes         500007
Ingtot            95450
Fex_c_y               0
Depto_y               0
Fex_dpto_y            0
Length: 157, dtype: int64


In [12]:
# Identificar columnas comunes entre train y test
common_columns = train_combined.columns.intersection(test_combined.columns)

# Actualizar las columnas numéricas y categóricas basadas en las columnas comunes
numeric_columns = train_combined[common_columns].select_dtypes(include=['float64', 'int64']).columns
categorical_columns = train_combined[common_columns].select_dtypes(include=['object']).columns

# Imputar columnas numéricas con la mediana
num_imputer = SimpleImputer(strategy='median')
train_combined[numeric_columns] = num_imputer.fit_transform(train_combined[numeric_columns])
test_combined[numeric_columns] = num_imputer.transform(test_combined[numeric_columns])  # Usar imputador ajustado

# Imputar columnas categóricas con la moda
cat_imputer = SimpleImputer(strategy='most_frequent')
train_combined[categorical_columns] = cat_imputer.fit_transform(train_combined[categorical_columns])
test_combined[categorical_columns] = cat_imputer.transform(test_combined[categorical_columns])  # Usar imputador ajustado

In [14]:
print("Valores faltantes restantes:\n", train_combined.isnull().sum().sum())
print("Valores faltantes restantes:\n", test_combined.isnull().sum().sum())

Valores faltantes restantes:
 0
Valores faltantes restantes:
 0
